# Simple Model Trainer

This notebook build datasets of indicator before using them to train 3 ML models. Thus we can compare them in order to choose the one that fit the most

In [1]:
%run stock_dataset_creator.ipynb

It's basically the data_viz notebook into a method using the sotcj 

In [40]:
def simple_model_trainer(name, lagmax = 5):
    stock_ds = stock(name=name)

    indicators = {"RSI" : stock_ds.RSI(), 
                      "MOM" : stock_ds.MOM(), 
                      "Bollinger Width" : stock_ds.Bollinger_width(), 
                      "ROC" : stock_ds.ROC(),
                      "Prices" : stock_ds.price()}
   
    data = pd.DataFrame(indicators)

    y = pd.DataFrame({"Prices" : stock_ds.price()})

    
    X = pd.DataFrame()
    for lag in range(1,lagmax):
        for col in data.columns:
            X[col + f"{lag} days ago"] = data[col].shift(lag)
    
        
    assert len(X)!=0, "X dataset is empty bordel"
    assert len(y)!= 0, "y dataset is empty bordel"

    X,y = X.dropna(), y.dropna()
    X_train = X["2014-01":"2016-01"]
    y_train = y["2014-01":"2016-01"]
    
    X_test_start = "2019-02"
    X_test_end = "2019-09"

    assert X_test_start < X_test_end

    X_test = X[X_test_start:X_test_end]
    y_test = y[X_test_start:X_test_end]


    if len(y_train)-len(X_train)>0 : y_train= y_train[:-abs(len(y_train)-len(X_train))]
    if len(y_train)-len(X_train)<0 : X_train= X_train[:-abs(len(y_train)-len(X_train))]


    from sklearn.svm import SVR
    from sklearn import linear_model
    
    models = {"SVR linear" : SVR(kernel="linear"), 
                "ridge " :linear_model.Ridge(alpha=.5), 
                "lasso": linear_model.Lasso(alpha=.1)}
        
    models_fitted = {model : models[model].fit(X_train, np.ravel(y_train)) for model in models.keys()}
    for model in models_fitted.keys() : 
        
        y_pred = models_fitted[model].predict(X_test)
        if len(y_test)-len(y_pred)>0 : 
            y_test= y_test[:-abs(len(y_test)-len(y_pred))]
            if len(y_test)-len(y_pred)!=0 : raise ValueError("L'ajustement ne fonctionne pas, y_test est trop long")
        if len(y_test)-len(y_pred)<0 : 
            y_pred= y_pred[:-abs(len(y_test)-len(y_pred))]
            if len(y_test)-len(y_pred)!=0 : raise ValueError("L'ajustement ne fonctionne pas, y_pred est trop long")
        y_test["pred_"+model] = y_pred
            
    return(y_test[X_test_end])

